# t5 Base model- fine tuning for Cover Letter Generation


In [2]:
import transformers
import pandas as pd
import evaluate

KeyboardInterrupt: 

### Dataset preparation, 
- The dataset was cleaned and separated into train and test splits in the dataset modelling file

In [ ]:
df_train = pd.read_parquet('../datasets/dataset3-train.parquet')
df_test = pd.read_parquet('../datasets/dataset3-test.parquet')

In [ ]:
df_train.sample(10)[1]['Cover Letter']

NameError: name 'df_train' is not defined

In [ ]:
# Column concatenation to result in feature and target cols
df_train['source_text'] = pd.concat([df_train['Job Title'].astype(str),df_train['Hiring Company'].astype(str),df_train['Applicant Name'].astype(str),df_train['Skillsets'].astype(str),df_train['Past Working Experience'].astype(str),df_train['Preferred Qualifications'].astype(str),df_train['Qualifications'].astype(str),df_train['Current Working Experience'].astype(str)],axis=1).agg(' '.join, axis=1)

df_test['source_text'] = pd.concat([df_test['Job Title'].astype(str),df_test['Hiring Company'].astype(str),df_test['Applicant Name'].astype(str),df_test['Skillsets'].astype(str),df_test['Past Working Experience'].astype(str),df_test['Preferred Qualifications'].astype(str),df_test['Qualifications'].astype(str),df_test['Current Working Experience'].astype(str)],axis=1).agg(' '.join, axis=1)

df_test.rename(columns={'Cover Letter': 'target_text'}, inplace=True)
df_train.rename(columns={'Cover Letter': 'target_text'}, inplace=True)

df_train = df_train[['source_text','target_text']]
df_test = df_test[['source_text','target_text']]

# T5 model expects a task related prefix
df_test['X'] = "generate: " + df_test['source_text']
df_train['X'] = "generate: " + df_train['source_text']

In [ ]:
df_test.head(1)

,source_text,target_text,X
0,Data Scientist XYZ Corporation John Smith Pyt...,"Dear Hiring Manager,\n\nI am writing to expres...",generate: Data Scientist XYZ Corporation John...


### Model import. t5-base was used as the starting point

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type='t5', model_name='t5-base')


### Model Trainning

In [ ]:
model.train(train_df=df_train,eval_df= df_test,source_max_token_len=1024,target_max_token_len=200,batch_size=8,use_gpu=False)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Missing logger folder: /Users/koechian/PycharmProjects/skillexplore/model/models/lightning_logs

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/koechian/anaconda3/envs/skillexplore/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
2024-01-04 13:43:48.034183: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Global seed set to 42
2024-01-04 13:43:59.571275: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild T

Training: 0it [00:00, ?it/s]

2024-01-04 13:44:38.734648: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Global seed set to 42
2024-01-04 13:44:50.041861: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Global seed set to 42
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingf